# Data cleaning

In [18]:
import pandas as pd
import numpy as np
import random
import json
from tensorflow.keras.models import Model, load_model
# import pycaret

# Replace double quotes with single quotes for moods JSON arrays
f = open('song_data.csv','r+')
text = f.read()
text = text.replace('"["', '"[?').replace('", "', '?,?').replace('"]"', '?]"') # Changed to "[$Happy$,$Sad$]" for easy replacing later
f.seek(0)
f.write(text)
f.close()

# Importing data
df = pd.read_csv('song_data.csv', index_col=0)
df.drop('uuid', axis=1, inplace=True)
df.dropna(inplace=True) # drop rows with nan values
for col in df.columns:
    if col not in ['id','isSkipped']:
        if col == 'moods':
            df[col] = df[col].apply(lambda x:x.replace('?','"')) # Replaces mood values back to the form ["Happy","Sad"] so it can be loaded by json
        df[col] = df[col].apply(json.loads)
df['activity'] = df.apply(lambda _: '', axis=1) # empty activity column
print('Number of samples: ', df.shape[0])
df.head()

Number of samples:  525


,gyroX,gyroY,gyroZ,accelX,accelY,accelZ,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,,,,,,,
1,"[499.30572509765625, 499.53460693359375, 0.198...","[1.861572265625, 2.49481201171875, 1.022338867...","[1.24359130859375, 1.2359619140625, 1.06048583...","[1.1943359375, 1.201171875, 1.1845703125, 1.18...","[1.1455078125, 1.1591796875, 1.1630859375, 1.1...","[3.68359375, 3.654296875, 3.6748046875, 3.6650...","[139.64, 138.36, 139.64, 140.28]","[30.50567626953125, 30.50567626953125, 30.5056...","[71.3134765625, 71.3134765625, 71.3134765625]","[Depressive, Atmospheric]",0,
2,"[498.1765747070313, 0.98419189453125, 1.579284...","[32.27996826171875, 14.7247314453125, 9.864807...","[497.9248046875, 496.368408203125, 494.9645996...","[2.9326171875, 2.9345703125, 2.728515625, 2.64...","[0.8466796875, 0.74609375, 0.865234375, 15.524...","[2.7548828125, 2.8037109375, 2.806640625, 3.34...","[123.24, 123.24, 139.32, 228.64]","[31.69403076171875, 31.69403076171875, 31.6940...","[67.05322265625, 67.05322265625, 67.0532226562...",[Depressive],0,
3,"[15.76995849609375, 10.65826416015625, 6.87408...","[488.36517333984375, 486.5798950195313, 496.92...","[3.86810302734375, 5.0811767578125, 498.947143...","[14.0107421875, 14.3212890625, 14.232421875, 1...","[14.96484375, 15.212890625, 15.275390625, 15.1...","[3.2021484375, 3.3291015625, 3.375, 3.35644531...","[256.08, 307.84000000000003, 315.2, 301.36, 30...","[32.21771240234375, 32.21771240234375, 32.2177...","[65.850830078125, 65.850830078125, 65.85083007...","[Passionate, Depressive]",0,
4,"[499.93896484375, 499.45068359375, 499.7482299...","[1.82342529296875, 2.74658203125, 1.8844604492...","[1.57928466796875, 1.434326171875, 1.365661621...","[1.7353515625, 1.708984375, 1.7333984375, 1.71...","[13.7841796875, 13.80078125, 13.7744140625, 13...","[2.8232421875, 2.8369140625, 2.8154296875, 2.8...","[127.08, 126.76, 125.48, 124.52]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...",[Elegant],1,
5,"[499.9465942382813, 0.03814697265625, 499.7482...","[2.01416015625, 1.77001953125, 1.7852783203125...","[1.1444091796875, 1.2359619140625, 1.129150390...","[0.4580078125, 0.4609375, 0.453125, 0.43847656...","[13.181640625, 13.1689453125, 13.1787109375, 1...","[2.6806640625, 2.693359375, 2.6875, 2.68652343...","[145.76, 144.48, 146.4, 144.8]","[32.42919921875, 32.42919921875, 32.4291992187...","[64.6728515625, 64.6728515625, 64.6728515625, ...","[Passionate, Depressive]",1,


In [19]:
data = df.iloc[0,:]
display(data)
# print(data['temp'])

gyroX        [499.30572509765625, 499.53460693359375, 0.198...
gyroY        [1.861572265625, 2.49481201171875, 1.022338867...
gyroZ        [1.24359130859375, 1.2359619140625, 1.06048583...
accelX       [1.1943359375, 1.201171875, 1.1845703125, 1.18...
accelY       [1.1455078125, 1.1591796875, 1.1630859375, 1.1...
accelZ       [3.68359375, 3.654296875, 3.6748046875, 3.6650...
optical                       [139.64, 138.36, 139.64, 140.28]
temp         [30.50567626953125, 30.50567626953125, 30.5056...
humidity         [71.3134765625, 71.3134765625, 71.3134765625]
moods                                [Depressive, Atmospheric]
isSkipped                                                    0
activity                                                      
Name: 1, dtype: object

In [20]:
# Filtering defective data

defective_ids = []
for idx,row in df.iterrows():
    # defective if temp array only has -40 values
    # defective if any humidity values are above 99.99
    if len([k for k in row['temp'] if k==-40]) == len(row['temp']) or \
    len([k for k in row['humidity'] if k>99]) == len(row['humidity']):
        defective_ids.append(idx)
    
    # if only some values are defective, keep the row, but remove defective values
    # remove -40 temp values and >99.99 humidity values
    elif (-40 in row['temp']) or len([k for k in row['humidity'] if k>99]) or (0 in row['optical']):
        df.at[idx,'temp'] = [k for k in row['temp'] if k!=-40]
        df.at[idx,'humidity'] = [k for k in row['humidity'] if k<=99]
        
    # some gyro/accel data have 40 samples
    # take the last 30 samples for these rows
    for col in df.columns[:6]:
        if len(row[col]) > 30:
            df.at[idx,col] = row[col][-30:]

filtered_df = df[~df.index.isin(defective_ids)].copy() # .copy() to avoid warning
print('%d defective rows: ' % len(defective_ids),defective_ids)

23 defective rows:  [21, 22, 33, 214, 236, 238, 245, 246, 247, 248, 249, 250, 251, 252, 386, 387, 388, 389, 390, 391, 392, 393, 394]


In [21]:
motion_model_path = 'firstModel_stackedLSTM.hd5'
model = load_model(motion_model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 128)           69120     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 64)            49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 100)               3300      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 134,547
Trainable params: 134,547
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Add activity from motion recognition model
x = [list(k) for k in filtered_df.iloc[:,:6].values]
x = np.array(x) # (num_samples, 6, 30)
x = np.array([k.T for k in x]) # reshape as (num_samples, 30, 6)

pred = model.predict(x)

activity_cats = np.array(['Running', 'Walking', 'Working']) # hardcoded categories from 'Physical Activity Classification.ipynb'
filtered_df['activity'] = activity_cats[np.argmax(pred, axis=1)]
#filtered_df['activity'] = np.argmax(pred, axis=1)
filtered_df = filtered_df.iloc[:,6:].copy() # drop gyro and accel columns

filtered_df.sample(5)

,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,
265,"[41.92, 40.64, 40.72, 40.56]","[29.2669677734375, 29.2669677734375, 29.266967...","[43.44482421875, 43.44482421875, 43.44482421875]",[Celebratory],0,Working
143,"[76.66, 12.16, 16.92, 23.12]","[31.85516357421875, 31.85516357421875, 31.8551...","[65.325927734375, 65.325927734375, 65.32592773...",[Elegant],0,Walking
350,"[42127.36, 55275.52, 58245.12, 74199.04000000001]","[32.2882080078125, 32.2882080078125, 32.288208...","[65.753173828125, 65.753173828125, 65.75317382...",[Warm],1,Walking
148,"[6645.76, 7326.72, 6681.6, 6635.52]","[31.3214111328125, 31.3214111328125, 31.321411...","[81.719970703125, 81.719970703125, 81.71997070...",[Warm],1,Walking
404,"[182.96, 184.56, 183.92, 183.6]","[32.47955322265625, 32.47955322265625, 32.4795...","[67.34619140625, 67.34619140625, 67.34619140625]",[Celebratory],1,Working


In [23]:
# Obtain mean optical, temp and humidity values

for col in filtered_df.columns:
    if col not in ['moods','isSkipped','activity']:
        filtered_df[col] = filtered_df[col].apply(np.mean)
filtered_df.head()

,optical,temp,humidity,moods,isSkipped,activity
id,,,,,,
1,139.480,30.505676,71.313477,"[Depressive, Atmospheric]",0,Working
2,153.610,31.694031,67.053223,[Depressive],0,Working
3,297.792,32.217712,65.850830,"[Passionate, Depressive]",0,Working
4,125.960,32.429199,64.672852,[Elegant],1,Working
5,145.360,32.429199,64.672852,"[Passionate, Depressive]",1,Working


In [24]:
# One-hot encoding for moods

moods = []
for k in filtered_df['moods'].values:
    moods += list(k)
moods = np.unique(np.array(moods))
for mood in moods:
    mood_values = filtered_df['moods'].astype(str).str.contains(mood)
    filtered_df[mood] = mood_values
filtered_df.drop('moods', axis=1, inplace=True)
print('Added one-hot encoded columns for moods:')
filtered_df.head()

Added one-hot encoded columns for moods:


,optical,temp,humidity,isSkipped,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm
id,,,,,,,,,,,,,
1,139.480,30.505676,71.313477,0,Working,False,False,True,False,True,False,False,False
2,153.610,31.694031,67.053223,0,Working,False,False,False,False,True,False,False,False
3,297.792,32.217712,65.850830,0,Working,False,False,False,False,True,False,True,False
4,125.960,32.429199,64.672852,1,Working,False,False,False,False,False,True,False,False
5,145.360,32.429199,64.672852,1,Working,False,False,False,False,True,False,True,False


In [25]:
# Invert mood boolean values based on "isSkipped"

ANTI_MOODS = ['Not Aggressive', 'Not Athletic', 'Not Atmospheric', 'Not Celebratory',\
             'Not Depressive', 'Not Elegant', 'Not Passionate', 'Not Warm']

filtered_df[ANTI_MOODS] = filtered_df[['Aggressive', 'Athletic', 'Atmospheric', 'Celebratory', \
                                       'Depressive', 'Elegant', 'Passionate', 'Warm']].copy()

for mood in moods:
    filtered_df[mood] = np.where(filtered_df['isSkipped'], 0, filtered_df[mood]) # remove isSkipped from normal decisions

for mood in ANTI_MOODS:
    filtered_df[mood] = np.where(filtered_df['isSkipped'], filtered_df[mood], 0) # and non-isSkipped from skip decisions

filtered_df.drop('isSkipped', axis=1, inplace=True)

print('Invert mood values based on "isSkipped" boolean:')
filtered_df.head()

Invert mood values based on "isSkipped" boolean:


,optical,temp,humidity,activity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,
1,139.480,30.505676,71.313477,Working,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,153.610,31.694031,67.053223,Working,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,297.792,32.217712,65.850830,Working,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,125.960,32.429199,64.672852,Working,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,145.360,32.429199,64.672852,Working,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [26]:
# One-hot encoding for activity
activities = []
for k in filtered_df['activity']:
    activities.append(k)
activities = np.unique(np.array(activities))
for activity in activities:
    activity_values = filtered_df['activity'].astype(str).str.contains(activity)
    activity_values = activity_values.astype(int)
    filtered_df[activity] = activity_values
filtered_df.drop('activity', axis=1, inplace=True)

filtered_df.head()

,optical,temp,humidity,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,...,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm,Running,Walking,Working
id,,,,,,,,,,,,,,,,,,,,,
1,139.480,30.505676,71.313477,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,153.610,31.694031,67.053223,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,297.792,32.217712,65.850830,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
4,125.960,32.429199,64.672852,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
5,145.360,32.429199,64.672852,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


# Split into train/test datasets

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# label encoding for activity
# le = LabelEncoder()
# filtered_df['activity'] = le.fit_transform(filtered_df['activity'].values)
# display(filtered_df.head())

# split into training & testing
x = filtered_df[['optical', 'temp', 'humidity','Working', 'Running', 'Walking']]
y = filtered_df[['Aggressive', 'Athletic', 'Atmospheric', 'Celebratory', \
                 'Depressive', 'Elegant', 'Passionate', 'Warm', \
                 'Not Aggressive', 'Not Athletic', 'Not Atmospheric', 'Not Celebratory',\
                 'Not Depressive', 'Not Elegant', 'Not Passionate', 'Not Warm']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

print('Training (x, y): ', x_train.shape, y_train.shape)
print('Testing (x, y): ', x_test.shape, y_test.shape)

# create dfs for training and test data for easy prediction later
train_df = x_train.join(y_train)
test_df = x_test.join(y_test)

print('\nTrain:')
display(train_df.head())
print('Test:')
display(test_df.head())

Training (x, y):  (401, 6) (401, 16)
Testing (x, y):  (101, 6) (101, 16)

Train:


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,...,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,
301,0.46,30.968933,35.168457,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
86,74.42,34.131165,75.988770,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
571,56.40,32.768250,67.089844,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,211.18,32.499695,63.629150,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
508,316.40,31.099854,79.947917,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


Test:


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,...,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
114,281.28,33.808899,59.564209,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
372,27973.12,38.139343,55.944824,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
379,1597.92,38.139343,55.944824,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
380,7883.84,38.139343,55.944824,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


# SVM

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import svm

# model training
# since predict_proba only works with two classes at a time,
# eight svm models are trained for each mood and stored in the dictionary "svm_moods"
svms_std = {} # key:mood, value:svm model trained on that mood
params = {'C':[0.001, 0.01, 0.1, 1, 10], 'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'decision_function_shape':('ovr', 'ovo')}
for mood in y_train.columns:
    svm_pipe_std = Pipeline([('scaler', StandardScaler()), 
                       ('svm', GridSearchCV(svm.SVC(max_iter=100000, probability=True), params)), ])
    svm_pipe_std.fit(x_train, y_train.loc[:,mood].values)
    svms_std[mood] = svm_pipe_std

c:\users\zeyu\desktop\internet of things\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\zeyu\desktop\internet of things\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\zeyu\desktop\internet of things\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
c:\users\zeyu\desktop\internet of things\venv\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100000).  

In [33]:
# Define functions for prediction and evaluation of SVM model

from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score

# Predict binary values or confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
def svm_predict(svms, input_data, prob=True):
    pred_df = input_data.copy()
    for mood,svm in svms.items():
        if prob:
            pred = svm.predict_proba(pred_df.iloc[:,:-16].values)
            pred_df[mood] = pred[:,1]
        else:
            pred = svm.predict(pred_df.iloc[:,:-16].values)
            pred_df[mood] = pred.reshape(-1)
    return pred_df
    
# Get loss (MSE) of predicted confidence scores
# Input: DataFrames of actual and predicted moods
def evaluate(df_actual, df_pred, df_pred_proba, clf):
    evaluation_df_dict = {
        'mood': [],
        'accuracy (%s)'%clf: [],
        'precision (%s)'%clf: [],
        'recall (%s)'%clf: [],
        'mse (%s)'%clf: [],
    }
    df_moods_actual = df_actual.iloc[:,-16:]
    df_moods_pred = df_pred.iloc[:,-16:]
    df_moods_pred_proba = df_pred_proba.iloc[:,-16:]
    for mood in df_moods_actual.columns:
        y_actual, y_pred, y_pred_proba = df_moods_actual[mood].values, df_moods_pred[mood].values, df_moods_pred_proba[mood].values
        evaluation_df_dict['mood'].append(mood)
        evaluation_df_dict['accuracy (%s)'%clf].append(accuracy_score(y_actual, y_pred))
        evaluation_df_dict['precision (%s)'%clf].append(precision_score(y_actual, y_pred, zero_division=0))
        evaluation_df_dict['recall (%s)'%clf].append(recall_score(y_actual, y_pred))
        evaluation_df_dict['mse (%s)'%clf].append(mean_squared_error(y_actual, y_pred_proba))
    evaluation_df = pd.DataFrame(evaluation_df_dict)
    evaluation_df.set_index('mood', inplace=True)
    return evaluation_df

In [49]:
train_pred_svm_df = svm_predict(svms_std, train_df, prob=False)
test_pred_svm_df = svm_predict(svms_std, test_df, prob=False)
train_prob_svm_df = svm_predict(svms_std, train_df, prob=True)
test_prob_svm_df = svm_predict(svms_std, test_df, prob=True)

# Show predicted values and confidence scores for moods for random samples
print('Test data mood labels (Actual):')
#display(test_df.head())

print('Test data mood labels (Predicted):')
#display(test_pred_svm_df.head())

print('Test data mood labels (Confidence Scores):')
display(test_prob_svm_df.head())

#Postprocess, normal categories - probabilities of skip categories

NORMAL_MOODS = ['Aggressive', 'Athletic', 'Atmospheric', 'Celebratory', \
                  'Depressive', 'Elegant', 'Passionate', 'Warm']

test_prob_svm_df[NORMAL_MOODS] = test_prob_svm_df[NORMAL_MOODS].to_numpy() - test_prob_svm_df[ANTI_MOODS].to_numpy()
test_prob_svm_df[ANTI_MOODS] = 0

print('Test data mood labels (Confidence Scores, Postprocessed):')
display(test_prob_svm_df.head())

# Show evaluation of predictions
train_eval_svm_df = evaluate(train_df, train_pred_svm_df, train_prob_svm_df, 'svm,std')
test_eval_svm_df = evaluate(test_df, test_pred_svm_df, test_prob_svm_df, 'svm,std')
display(test_eval_svm_df)
display(test_eval_svm_df.describe())

Test data mood labels (Actual):
Test data mood labels (Predicted):
Test data mood labels (Confidence Scores):


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,...,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,0.047588,0.084840,0.078369,0.073572,...,0.135455,0.143277,0.127595,0.011879,0.084889,0.036015,0.053410,0.056104,0.098115,0.087789
114,281.28,33.808899,59.564209,1,0,0,0.045362,0.081715,0.079233,0.073423,...,0.135633,0.139980,0.162444,0.008122,0.084831,0.036047,0.044937,0.044181,0.100905,0.087950
372,27973.12,38.139343,55.944824,0,0,1,0.044120,0.056693,0.119902,0.245689,...,0.138017,0.056821,0.074994,0.012369,0.084328,0.036070,0.096450,0.077247,0.103274,0.080669
379,1597.92,38.139343,55.944824,0,0,1,0.036161,0.061859,0.095053,0.143846,...,0.137685,0.097850,0.288151,0.007866,0.084401,0.035999,0.154878,0.085455,0.126182,0.084215
380,7883.84,38.139343,55.944824,0,0,1,0.037920,0.059998,0.100520,0.038660,...,0.137764,0.085337,0.216257,0.008762,0.084384,0.036016,0.139791,0.084780,0.120356,0.083357


Test data mood labels (Confidence Scores, Postprocessed):


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,...,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,-0.080007,0.072961,-0.006520,0.037557,...,0.037340,0.055488,0,0,0,0,0,0,0,0
114,281.28,33.808899,59.564209,1,0,0,-0.117082,0.073593,-0.005598,0.037377,...,0.034728,0.052030,0,0,0,0,0,0,0,0
372,27973.12,38.139343,55.944824,0,0,1,-0.030874,0.044324,0.035574,0.209619,...,0.034743,-0.023849,0,0,0,0,0,0,0,0
379,1597.92,38.139343,55.944824,0,0,1,-0.251990,0.053993,0.010651,0.107847,...,0.011503,0.013635,0,0,0,0,0,0,0,0
380,7883.84,38.139343,55.944824,0,0,1,-0.178337,0.051235,0.016136,0.002644,...,0.017408,0.001980,0,0,0,0,0,0,0,0


,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)"
mood,,,,
Aggressive,0.960396,0.0,0.000000,0.045741
Athletic,0.900990,0.0,0.000000,0.092102
Atmospheric,0.841584,0.0,0.000000,0.159325
Celebratory,0.861386,0.5,0.071429,0.130599
Depressive,0.920792,0.0,0.000000,0.072417
Elegant,0.891089,0.0,0.000000,0.105148
Passionate,0.920792,0.0,0.000000,0.074833
Warm,0.910891,0.0,0.000000,0.083960
Not Aggressive,0.940594,0.0,0.000000,0.059406


,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)"
count,16.000000,16.00000,16.000000,16.000000
mean,0.915223,0.03125,0.004464,0.078080
std,0.036148,0.12500,0.017857,0.033144
min,0.841584,0.00000,0.000000,0.029703
25%,0.891089,0.00000,0.000000,0.059406
50%,0.920792,0.00000,0.000000,0.073625
75%,0.940594,0.00000,0.000000,0.089857
max,0.970297,0.50000,0.071429,0.159325


# Random Forest

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

params = {
    'criterion':('gini', 'entropy'),
    'max_depth': list(range(1,21)),
    'min_samples_split': (4,24)
}

# About the same, can swap back to try. DecisionTreeClassifier much faster
#rf = GridSearchCV(RandomForestClassifier(), params)
rf = GridSearchCV(DecisionTreeClassifier(max_features=6), params)
rf.fit(x_train, y_train)
rf.best_params_

{'criterion': 'gini', 'max_depth': 19, 'min_samples_split': 4}

In [52]:
# Predict binary values or confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
# Returns confidence scores if prob=True else binary values
def rf_predict(rf, input_data, prob=True):
    pred_df = input_data.copy()
    data = input_data.iloc[:,:-16]
    target = input_data.iloc[:,-16:]
    if prob:
        pred = np.array(rf.predict_proba(data.values))
    else:
        pred = np.array(rf.predict(data.values))
    for i in range(len(target.columns)):
        mood = target.columns[i]
        if prob:
            pred_df[mood] = pred[i,:,1] # predict_proba returns shape (n_features, n_samples, probs)
        else:
            pred_df[mood] = pred[:,i] # predict returns shape (n_samples, n_features)
    return pred_df

In [63]:
# Predict binary values or confidence scores for moods from sensor data
# For now, input_data is a Dataframe for convenience
# Returns confidence scores if prob=True else binary values

train_pred_rf_df = rf_predict(rf, train_df, prob=False)
test_pred_rf_df = rf_predict(rf, test_df, prob=False)
train_prob_rf_df = rf_predict(rf, train_df, prob=True)
test_prob_rf_df = rf_predict(rf, test_df, prob=True)

# Show predicted values and confidence scores for moods for random samples
print('Test data mood labels (Actual):')
#test_df_copy = test_df.copy()
#test_df_copy['activity'] = [activity_cats[k] for k in test_df_copy['activity'].values]
# test_df_copy['activity'] = le.inverse_transform(test_df_copy['activity'].values) # convert activity values back to strings
display(test_df.head(10))

print('Test data mood labels (Predicted):')
# test_pred_rf_df['activity'] = le.inverse_transform(test_pred_rf_df['activity'].values)
#test_pred_rf_df['activity'] = [activity_cats[k] for k in test_pred_rf_df['activity'].values]
display(test_pred_rf_df.head(10))

print('Test data mood labels (Confidence Scores):')
#test_prob_rf_df['activity'] = [activity_cats[k] for k in test_prob_rf_df['activity'].values]
# test_pred_proba_rf_df['activity'] = le.inverse_transform(test_pred_proba_rf_df['activity'].values)
pd.set_option('display.max_columns', None)
display(test_prob_rf_df.head(10))


test_prob_rf_df[NORMAL_MOODS] = test_prob_rf_df[NORMAL_MOODS].to_numpy() - test_prob_rf_df[ANTI_MOODS].to_numpy()
test_prob_rf_df[ANTI_MOODS] = 0

print('Test data mood labels (Confidence Scores, Postprocessed):')
display(test_prob_rf_df.head(10))

# Show evaluation of predictions
train_eval_rf_df = evaluate(train_df, train_pred_rf_df, train_prob_rf_df, 'rf')
test_eval_rf_df = evaluate(test_df, test_pred_rf_df, test_prob_rf_df, 'rf')
display(test_eval_rf_df)
display(test_eval_rf_df.describe())


Test data mood labels (Actual):


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
114,281.28,33.808899,59.564209,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
372,27973.12,38.139343,55.944824,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
379,1597.92,38.139343,55.944824,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
380,7883.84,38.139343,55.944824,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
16,240.84,32.677612,62.847900,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
358,5570.88,31.039429,66.339111,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
491,48.60,32.802826,59.768677,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
180,2789.28,33.255005,80.975342,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Test data mood labels (Predicted):


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
114,281.28,33.808899,59.564209,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
372,27973.12,38.139343,55.944824,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
379,1597.92,38.139343,55.944824,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
380,7883.84,38.139343,55.944824,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
16,240.84,32.677612,62.847900,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
358,5570.88,31.039429,66.339111,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
491,48.60,32.802826,59.768677,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
180,2789.28,33.255005,80.975342,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Test data mood labels (Confidence Scores):


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.666667,0.0,0.666667,0.000000
114,281.28,33.808899,59.564209,1,0,0,0.000000,0.666667,0.0,0.666667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
372,27973.12,38.139343,55.944824,0,0,1,0.000000,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
379,1597.92,38.139343,55.944824,0,0,1,0.333333,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.333333,0.000000,0.333333,0.0,0.333333,0.0,0.000000,0.000000
380,7883.84,38.139343,55.944824,0,0,1,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
16,240.84,32.677612,62.847900,1,0,0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.333333
358,5570.88,31.039429,66.339111,0,0,1,0.000000,0.666667,0.0,0.000000,0.0,0.0,0.333333,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
491,48.60,32.802826,59.768677,1,0,0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.666667,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
180,2789.28,33.255005,80.975342,0,1,0,0.000000,1.000000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000


Test data mood labels (Confidence Scores, Postprocessed):


,optical,temp,humidity,Working,Running,Walking,Aggressive,Athletic,Atmospheric,Celebratory,Depressive,Elegant,Passionate,Warm,Not Aggressive,Not Athletic,Not Atmospheric,Not Celebratory,Not Depressive,Not Elegant,Not Passionate,Not Warm
id,,,,,,,,,,,,,,,,,,,,,,
107,376.96,32.922668,61.926270,1,0,0,0.000000,-0.333333,0.000000,0.000000,-0.666667,0.0,-0.666667,0.000000,0,0,0,0,0,0,0,0
114,281.28,33.808899,59.564209,1,0,0,0.000000,0.666667,0.000000,0.666667,0.000000,0.0,0.000000,0.000000,0,0,0,0,0,0,0,0
372,27973.12,38.139343,55.944824,0,0,1,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0,0,0,0,0,0,0,0
379,1597.92,38.139343,55.944824,0,0,1,0.000000,0.000000,-0.333333,0.000000,-0.333333,0.0,0.000000,0.000000,0,0,0,0,0,0,0,0
380,7883.84,38.139343,55.944824,0,0,1,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0,0,0,0,0,0,0,0
16,240.84,32.677612,62.847900,1,0,0,-0.666667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.333333,0,0,0,0,0,0,0,0
358,5570.88,31.039429,66.339111,0,0,1,0.000000,0.666667,0.000000,0.000000,0.000000,0.0,0.333333,0.333333,0,0,0,0,0,0,0,0
491,48.60,32.802826,59.768677,1,0,0,-0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.666667,0,0,0,0,0,0,0,0
180,2789.28,33.255005,80.975342,0,1,0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0,0,0,0,0,0,0,0


,accuracy (rf),precision (rf),recall (rf),mse (rf)
mood,,,,
Aggressive,0.940594,0.000000,0.000000,0.137514
Athletic,0.851485,0.272727,0.300000,0.143839
Atmospheric,0.782178,0.200000,0.125000,0.295930
Celebratory,0.792079,0.230769,0.214286,0.227998
Depressive,0.881188,0.000000,0.000000,0.175468
Elegant,0.821782,0.000000,0.000000,0.209296
Passionate,0.861386,0.000000,0.000000,0.175468
Warm,0.792079,0.000000,0.000000,0.238724
Not Aggressive,0.881188,0.125000,0.166667,0.059406


,accuracy (rf),precision (rf),recall (rf),mse (rf)
count,16.000000,16.000000,16.000000,16.000000
mean,0.870668,0.069638,0.065997,0.130586
std,0.052822,0.111744,0.107123,0.081951
min,0.782178,0.000000,0.000000,0.029703
25%,0.844059,0.000000,0.000000,0.059406
50%,0.876238,0.000000,0.000000,0.113311
75%,0.896040,0.143750,0.135417,0.183925
max,0.950495,0.285714,0.300000,0.295930


In [35]:
evaluate_df = pd.concat([test_eval_svm_df, test_eval_rf_df], axis=1)
display(evaluate_df)
display(evaluate_df.describe())

,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)",accuracy (rf),precision (rf),recall (rf),mse (rf)
mood,,,,,,,,
Aggressive,0.896552,0.0,0.0,0.098554,0.896552,0.000000,0.000000,0.114033
Athletic,0.844828,0.0,0.0,0.131442,0.810345,0.000000,0.000000,0.126039
Atmospheric,0.775862,0.0,0.0,0.181373,0.758621,0.000000,0.000000,0.202016
Celebratory,0.827586,0.0,0.0,0.142576,0.810345,0.000000,0.000000,0.162678
Depressive,0.827586,0.0,0.0,0.143697,0.793103,0.000000,0.000000,0.150250
Elegant,0.862069,0.0,0.0,0.119464,0.793103,0.000000,0.000000,0.168010
Passionate,0.775862,0.0,0.0,0.175535,0.741379,0.333333,0.153846,0.193339
Warm,0.793103,0.0,0.0,0.163203,0.741379,0.000000,0.000000,0.213863


,"accuracy (svm,std)","precision (svm,std)","recall (svm,std)","mse (svm,std)",accuracy (rf),precision (rf),recall (rf),mse (rf)
count,8.000000,8.0,8.0,8.000000,8.000000,8.000000,8.000000,8.000000
mean,0.825431,0.0,0.0,0.144480,0.793103,0.041667,0.019231,0.166279
std,0.042670,0.0,0.0,0.028247,0.050478,0.117851,0.054393,0.035648
min,0.775862,0.0,0.0,0.098554,0.741379,0.000000,0.000000,0.114033
25%,0.788793,0.0,0.0,0.128448,0.754310,0.000000,0.000000,0.144197
50%,0.827586,0.0,0.0,0.143136,0.793103,0.000000,0.000000,0.165344
75%,0.849138,0.0,0.0,0.166286,0.810345,0.000000,0.000000,0.195508
max,0.896552,0.0,0.0,0.181373,0.896552,0.333333,0.153846,0.213863


## Save Model

In [32]:
import pickle
pickle.dump(rf, open('RandomForest', 'wb'))